# Module 2 - Amazon Q cross-app index

### Use Amazon Q Business cross-app index to search relevant content

In this module, we will call the Amazon Q Business `SearchRelevantContent` API on the cross-app index to perform a semantic and keyword (Hybrid) based search. This search is useful for building Retrieval Augmented Generation (RAG) based generative AI applications. We will review the response of the API which will return textual chunks of relevant data as per a user query. We will then use these relevant chunks of information to perform a RAG based Q&A with Anthropic Claude Sonnet 3.5 model via Amazon Bedrock.

Before we proceed further, let's install some pre-requisites first. Run the following code-block.

<div class="alert alert-block alert-warning"> <b>IMPORTANT:</b> Please make sure that you select <b>"Python 3 (ipykernel)"</b> kernel for this notebook from the top right corner, if one is not selected already. </div>

## Step 1

In [ ]:
!python -m pip install boto3


#### Pre-requisites

In order to be able to call Amazon Q Business's data APIs, we will need to acquire credentials that are tagged to a specific user id (in this case, an email address). We have pre-deployed a mechanism that will help generate this credentials with a helper function. First we will need to acquire some necessary details that will help us generate the identity-aware SigV4 AWS credentials. Specifically, we will require the following details.

- `issuer`: the issuer URL
- `client_id`: a client_id for the OIDC client
- `client_secret`: a client_secret for the OIDC client
- `role_arn`: the IAM role to assume, this is the role that has Amazon Q Business permissions
- `region`: `us-east-1` this is the current region where our Amazon Q Business Application is setup 
- `email`: your email address (can be a fictional email address of the format user@email.com)

To obtain the values, execute the next code cell which will read a JSON file from an S3 bucket.

## Step 2

In [ ]:
import boto3
import json
import sagemaker

s3_client = boto3.client('s3')
role = sagemaker.get_execution_role()
account_id = role.split(':')[4]
bucket_name = f'amazon-q-tvm-{account_id}'
file_key = 'tvm_values.json'

response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
creds = json.loads(response['Body'].read().decode('utf-8'))["TVMOidcIssuerStack"]
creds["IssuerUrlOutput"] = creds["IssuerUrlOutput"].rstrip('/')

We will use the JSON data above to initialize our helper function `TVMClient` in the next code block. Go ahead and enter your email address in the `email_address` variable if you wish to, else you can keep `jon_doe@email.com`. Once done, execute the code block.

## Step 3

In [ ]:
from utils.tvm_client import TVMClient

email_address="john_doe@email.com"

token_client = TVMClient(
        issuer=creds["IssuerUrlOutput"],
        client_id=creds["QbizTVMClientID"],
        client_secret=creds["QbizTVMClientSecret"],
        role_arn=creds["QBizAssumeRoleARN"],
        region="us-east-1"
)
    
# Get Sigv4 credentials using TVM
credentials = token_client.get_sigv4_credentials(email=email_address)

## Step 4

_Use SigV4 credentials to initialize Amazon Q Business Client_: We will then initialize an Amazon Q Business Boto3 (Python) client with the SigV4 credentials obtained using the helper function to make calls to Amazon Q Business APIs (in this case the `SearchRelevantContent` API).

In [ ]:
import boto3

qbiz = boto3.client("qbusiness", **credentials)
        

## Calling `SearchRelevantContent` API
---

In the previous code block, we initialized an Amazon Q Business client with the required credentials. We are now ready to make the call to the `SearchRelevantContent` API and analyze it's response. To call the API, we require the Amazon Q Business Application id (`applicationId`) and the retriever ID (`retrieverId`). 

Execute the next code block under "Step 5" which will fetch the application ID and retriever ID respectively.

## Step 5

In [ ]:
Q_BIZ_APP_ID = ""
Q_RETRIEVER_ID = ""

import boto3
client = boto3.client('qbusiness')
response_app = client.list_applications()

for r in response_app["applications"]:
    if 'aim333-module-2' in r['displayName']:
        Q_BIZ_APP_ID=r['applicationId']

if Q_BIZ_APP_ID:
    response_ret = client.list_retrievers(applicationId=Q_BIZ_APP_ID)
    Q_RETRIEVER_ID = response_ret['retrievers'][0]['retrieverId']

print(f"Application ID is: {Q_BIZ_APP_ID}\nRetriever ID is: {Q_RETRIEVER_ID}")

<div class="alert alert-block alert-info"> 
    <b><i>(Optional)</i> View Application ID and Retriever ID in the Amazon Q Business console:</b> 
    <p>
    You can also get the values for application ID and retriever ID from the Amazon Q Business console under the application named `aim333-module-2`. Follow these steps to view the values (this is informational only, if you have executed the code block above you may move on to the next step) -</p>

<ol>
    <li> Navigate to the Amazon Q Business console.</li>
    <li> Click on "Applications" from the menu on the left.</li>
    <li> Click on the application named `aim333-module-2`.</li>
    <li> In the following screen, scroll down to the "Application Settings" section.</li>
    <li> The application ID is found under the label "Application ID".</li>    
</ol>
<p>Next, to get the retriever ID</p>
<ol>
    <li> In the same screen, click on the next tab labeled "Index".</li>
    <li> The retriever ID is found under the label "Retriever ID".</li>
</ol>
</div>

Let's test this client by calling the `ChatSync` API first. At this stage our user `john_doe@email.com` is not subscribed to the Amazon Q Business Application. The call to `ChatSync` API will auto subscribe the user with an `AccessDeniedException`, this is normal and a one-time action after which the `ChatSync` operation can be retried.

In [ ]:
import time

chat_params = {
    "applicationId": Q_BIZ_APP_ID,
    "userMessage": "What are the reasons for keyboard failures?"
}

def call_chat_sync(chat_params):
    try:
        response = qbiz.chat_sync(**chat_params)
        print(f"Answer: {response['systemMessage']}")
        print("=========Sources=========")
        for source in response['sourceAttributions']:
            print(f'Title: {source["title"]}, URL: {source["url"]}')
    except qbiz.exceptions.AccessDeniedException:
        print("User subscribed...\n\n")
        time.sleep(2)
        call_chat_sync(chat_params)
    except Exception as err:
        print(err)
        print("Please retry")
        
call_chat_sync(chat_params)

Next we will run the `SearchRelevantContent` API.

In [ ]:
import boto3
import pprint
qbiz = boto3.client("qbusiness", region_name="us-east-1", **credentials)


search_params = {  'applicationId': Q_BIZ_APP_ID, 
    'contentSource': {
        'retriever': { 
            'retrieverId': Q_RETRIEVER_ID 
            }
    }, 
    'queryText': 'What are the reasons for keyboard failures?', 
    'maxResults': 5
}

def call_search_relevant_content(search_params):
    try:
        search_response = qbiz.search_relevant_content(**search_params)
        pprint.pprint(search_response['relevantContent'])
        return search_response['relevantContent']
    except Exception as e:
        print(e)

relevant_content = call_search_relevant_content(search_params)

## Build a RAG application with `SearchRelevantContent`

Next, we will build a RAG (Retrieval Augmented Generation) application with results of `SearchRelevantContent` API. Here, we will use Amazon Bedrock models and APIs to perform Q&A on the relevant data retrieved by the `SearchRelevantContent` API. Let's initialize Amazon Bedrock Client.

**NOTE**: You must have completed the pre-requisites in the beginning of the workshop to enable Anthropic Claude Sonnet 3.5 model access in order to proceed with this section of the hands-on. Also note that we are specifically accessing Bedrock in `us-west-2` region.

## Step 6

In [ ]:
import boto3

bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')

We will be specifically using Amazon Bedrock's `converse` API to call the model. Let's invoke the model with a sample SYSTEM_PROMPT and query, with no context.

## Step 7

In [ ]:
QUERY="What is the typical reason of company keyboards not working?"

SYSTEM_PROMPT=""""
You are a helpful AI assistant who answers question correctly and accurately about a AcmeCompany's IT tickets. Do not makeup answers and only answer from the provided knowledge.
"""

messages = [
    {
        "role": "user",
        "content":[
            {"text": QUERY}
        ]
    }
]
converse_params = {
        "modelId": 'anthropic.claude-3-5-sonnet-20240620-v1:0',
        "messages": messages,                
        "system": [{"text": SYSTEM_PROMPT}]
    }
ai_response = bedrock_client.converse(**converse_params)

print(ai_response['output']['message']['content'][0]['text'])

As you can see, the model is unable to answer this question because it has no context related to AcmeCompany's IT support tickets. Now let's use the results from `SearchRelevantContent` API to build a context and provide it to the model. We will make some code changes to better prompt the model with the additional context. But before we do that, lets gather all the `content` pieces that we got via the `SearchRelevantContent` API.

## Step 8

In [ ]:
full_context = ""

for chunks in relevant_content:
    full_context = full_context + chunks['content'] + "\n"

print(full_context)

Now that we have the full context and any relevant information related to the `QUERY` we are ready to provide this information to the model and ask our question again.

## Step 9

In [ ]:
QUERY="What is the typical reason of company keyboards not working?"

SYSTEM_PROMPT=""""
You are a helpful AI assistant who answers question correctly and accurately about a AcmeCompany's IT tickets. Do not makeup answers and only answer from the provided knowledge.
"""

messages = [{"role": "user","content":[{"text": f"Given the full context: {full_context}\n\nAnswer this question accurately: {QUERY}"}]}]
converse_params = {
        "modelId": 'anthropic.claude-3-5-sonnet-20240620-v1:0',
        "messages": messages,                
        "system": [{"text": SYSTEM_PROMPT}]
    }
ai_response = bedrock_client.converse(**converse_params)

print(ai_response['output']['message']['content'][0]['text'])

# Conclusion
---

In this module we learnt how to call Amazon Q Business cross-app index which gives you access to data from variaous enterprise applications from across the organization and your customer's organization. We then saw how to use the results returned by the `SearchRelevantContent` API to build powerful RAG-based Q&A applications with any LLM. In our example, we saw how the model was unable to answer the question without much context about the question. Once the full context relevan't to the user `QUERY` was provided, thanks to the cross-app index, the model's response was much better and accurate.

This concludes our workshop! Thanks for joining us and please take a moment to fill out the Survey for Session ID: AIM333 in the "AWS Events" mobile App or by scanning the QR code.